# Process all file in folder





In [1]:
%load_ext autoreload
%autoreload 2

List the files to process in both conditions wt and sega

In [ ]:
import pandas as pd

from pathlib import Path

# list files from the src folder
src = Path('/media/cephfs2/jparham/Joe for Jerome /')
dst = Path('./res')

filelist = pd.read_csv('filelist.csv', index_col=0)
print(f"Number of files {len(filelist)}")
filelist.head()


Number of files 62


,path,name,condition
0,/media/cephfs2/jparham/Joe for Jerome /SegA Li...,01a_delSegA_CM_SyS_reg_Cell_0.tif,sega
1,/media/cephfs2/jparham/Joe for Jerome /SegA Li...,01a_delSegA_CM_SyS_reg_Cell_12.tif,sega
2,/media/cephfs2/jparham/Joe for Jerome /SegA Li...,01a_delSegA_CM_SyS_reg_Cell_13.tif,sega
3,/media/cephfs2/jparham/Joe for Jerome /SegA Li...,01a_delSegA_CM_SyS_reg_Cell_3.tif,sega
4,/media/cephfs2/jparham/Joe for Jerome /SegA Li...,01a_delSegA_CM_SyS_reg_Cell_11.tif,sega


## Process all file

In [3]:
# from tqdm.notebook import tqdm
# tqdm.pandas()
# import motionquant as mq

# results_path = dst/Path('results.h5') # result
# if results_path.exists():
#     results_path.unlink()

# print(f"Saving results in {results_path}")

# df = []
# for filename in tqdm(filelist['path'].iloc):
#     try:
#         name = filename.stem        
#         img, mask, position, speed, diff, flow, rho, div, blob = mq.process(filename)
#         df.append(mq.record(name, img, mask, position, speed, diff, flow, rho, div, blob))
#         mq.save_result(results_path, name, img, mask, position, speed, diff, flow, rho, div, blob)
#     except Exception as e:
#         print(f"file '{filename}' could not be processed")
#         print(e)
#         pass
# df = pd.concat(df)


## Parallel processing

In [4]:
from dask.distributed import LocalCluster
cluster = LocalCluster()
client = cluster.get_client()
cluster.scale(6)
cluster

LocalCluster(13309ec9, 'tcp://127.0.0.1:37233', workers=4, threads=12, memory=31.27 GiB)

Create a tag for the result

In [5]:
import git
repo = git.Repo(search_parent_directories=True)
tag = repo.head.object.hexsha[-6:]
print(tag)

503b9c


In [14]:
process(filelist.iloc[0]['path'], results_path)

/home/jeromeb/micromamba/envs/imaging/lib/python3.11/site-packages/cellpose/resnet_torch.py:276: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(filena

['y', 'x']


FileNotFoundError: [Errno 2] Unable to synchronously create file (unable to open file: name = 'res/results-503b9c.h5', errno = 2, error message = 'No such file or directory', flags = 15, o_flags = c2)

In [15]:
import motionquant as mq
import dask
from dask.distributed import Lock
import traceback

def process(filename, results_path):
    """Process files and save results in a hf5 file"""
    name = filename.stem        
    img, mask, position, speed, diff, flow, rho, div, blob = mq.process(Path(filename))
    lock = Lock('process-sufo')
    lock.acquire()
    mq.save_result(results_path, name, img, mask, position, speed, diff, flow, rho, div, blob)
    lock.release()
    df = mq.record(name, img, mask, position, speed, diff, flow, rho, div, blob)
    return df

def process_safe(filename, results_path):
    """Process files catching exceptions"""
    try:
        return process(filename, results_path)
    except Exception as e:
        print(f"file '{filename}' could not be processed")
        print(e)
        print(traceback.print_exc())
        pass

results_path = dst/Path(f'results-{tag}.h5') # result
if results_path.exists():
    results_path.unlink()

print(f"Saving results in file '{results_path}'")
tsk = [dask.delayed(process_safe)(filename, results_path) for filename in filelist['path'].iloc]
res = dask.compute(tsk)
df = pd.concat(res[0])

Saving results in file 'res/results-503b9c.h5'


TypeError: expected str, bytes or os.PathLike object, not _iLocIndexer

file '/media/cephfs2/jparham/Joe for Jerome /Divisions with G2toConstriction/01a_DSM639_SyS_CM_start_reg_Cell_24.tif' could not be processed
No matching signature found
None


Traceback (most recent call last):
  File "/tmp/ipykernel_141074/1493789395.py", line 20, in process_safe
  File "/tmp/ipykernel_141074/1493789395.py", line 9, in process
  File "/home/jeromeb/work/code/baum/dna-movement-sufolobus/motionquant.py", line 435, in process
    blob_labels, blobs_trj = segment_and_track_dna_blobs(pimg[:, 1], cell_mask)
                             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/jeromeb/work/code/baum/dna-movement-sufolobus/motionquant.py", line 248, in segment_and_track_dna_blobs
    blob = np.stack([segment_watershed(b, 5.0) for b in blob])
                    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/jeromeb/work/code/baum/dna-movement-sufolobus/motionquant.py", line 248, in <listcomp>
    blob = np.stack([segment_watershed(b, 5.0) for b in blob])
                     ^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/jeromeb/work/code/baum/dna-movement-sufolobus/motionquant.py", line 30, in segment_watershed
    labels = s

file '/media/cephfs2/jparham/Joe for Jerome /Divisions with G2toConstriction/01a_DSM639_SyS_CM_start_reg_Cell_15.tif' could not be processed
No matching signature found
None


...

Traceback (most recent call last):
  File "/tmp/ipykernel_141074/1493789395.py", line 20, in process_safe
  File "/tmp/ipykernel_141074/1493789395.py", line 9, in process
  File "/home/jeromeb/work/code/baum/dna-movement-sufolobus/motionquant.py", line 435, in process
    blob_labels, blobs_trj = segment_and_track_dna_blobs(pimg[:, 1], cell_mask)
                             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/jeromeb/work/code/baum/dna-movement-sufolobus/motionquant.py", line 248, in segment_and_track_dna_blobs
    blob = np.stack([segment_watershed(b, 5.0) for b in blob])
                    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/jeromeb/work/code/baum/dna-movement-sufolobus/motionquant.py", line 248, in <listcomp>
    blob = np.stack([segment_watershed(b, 5.0) for b in blob])
                     ^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/jeromeb/work/code/baum/dna-movement-sufolobus/motionquant.py", line 30, in segment_watershed
    labels = s

file '/media/cephfs2/jparham/Joe for Jerome /SegA Live imaging /20240229_delSegA/CroppedDivisions/01a_delSegA_CM_SyS_reg_Cell_13.tif' could not be processed
No matching signature found
None


Traceback (most recent call last):
  File "/tmp/ipykernel_141074/1493789395.py", line 20, in process_safe
  File "/tmp/ipykernel_141074/1493789395.py", line 9, in process
  File "/home/jeromeb/work/code/baum/dna-movement-sufolobus/motionquant.py", line 435, in process
    blob_labels, blobs_trj = segment_and_track_dna_blobs(pimg[:, 1], cell_mask)
                             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/jeromeb/work/code/baum/dna-movement-sufolobus/motionquant.py", line 248, in segment_and_track_dna_blobs
    blob = np.stack([segment_watershed(b, 5.0) for b in blob])
                    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/jeromeb/work/code/baum/dna-movement-sufolobus/motionquant.py", line 248, in <listcomp>
    blob = np.stack([segment_watershed(b, 5.0) for b in blob])
                     ^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/jeromeb/work/code/baum/dna-movement-sufolobus/motionquant.py", line 30, in segment_watershed
    labels = s

file '/media/cephfs2/jparham/Joe for Jerome /SegA Live imaging /20240229_delSegA/CroppedDivisions/01a_delSegA_CM_SyS_reg_Cell_7.tif' could not be processed
No matching signature found
None


Traceback (most recent call last):
  File "/tmp/ipykernel_141074/1493789395.py", line 20, in process_safe
  File "/tmp/ipykernel_141074/1493789395.py", line 9, in process
  File "/home/jeromeb/work/code/baum/dna-movement-sufolobus/motionquant.py", line 435, in process
    blob_labels, blobs_trj = segment_and_track_dna_blobs(pimg[:, 1], cell_mask)
                             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/jeromeb/work/code/baum/dna-movement-sufolobus/motionquant.py", line 248, in segment_and_track_dna_blobs
    blob = np.stack([segment_watershed(b, 5.0) for b in blob])
                    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/jeromeb/work/code/baum/dna-movement-sufolobus/motionquant.py", line 248, in <listcomp>
    blob = np.stack([segment_watershed(b, 5.0) for b in blob])
                     ^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/jeromeb/work/code/baum/dna-movement-sufolobus/motionquant.py", line 30, in segment_watershed
    labels = s

In [7]:
df.to_csv(dst/'results.csv')